Import Libraries

In [2]:
import numpy as np 
import pandas as pd 
import os

import matplotlib.pyplot as plt
import csv
import itertools
import collections

import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed

%matplotlib inline

Read files

In [12]:
path = './MIT-BIH/';
# Read files
filenames = next(os.walk(path))[2]

# Split and save .csv , .txt 
records = list()
annotations = list()
filenames.sort()

In [4]:
# segrefating filenames and annotations
for f in filenames:
    filename, file_extension = os.path.splitext(f)
    
    # *.csv
    if(file_extension == '.csv'):
        records.append(path + filename + file_extension)

    # *.txt
    else:
        annotations.append(path + filename + file_extension)
print(records)

['./MIT-BIH/100_MLII.csv', './MIT-BIH/100_V5.csv', './MIT-BIH/101_MLII.csv', './MIT-BIH/101_V1.csv', './MIT-BIH/102_V2.csv', './MIT-BIH/102_V5.csv', './MIT-BIH/103_MLII.csv', './MIT-BIH/103_V2.csv', './MIT-BIH/104_V2.csv', './MIT-BIH/104_V5.csv', './MIT-BIH/105_MLII.csv', './MIT-BIH/105_V1.csv', './MIT-BIH/106_MLII.csv', './MIT-BIH/106_V1.csv', './MIT-BIH/107_MLII.csv', './MIT-BIH/107_V1.csv', './MIT-BIH/108_MLII.csv', './MIT-BIH/108_V1.csv', './MIT-BIH/109_MLII.csv', './MIT-BIH/109_V1.csv', './MIT-BIH/111_MLII.csv', './MIT-BIH/111_V1.csv', './MIT-BIH/112_MLII.csv', './MIT-BIH/112_V1.csv', './MIT-BIH/113_MLII.csv', './MIT-BIH/113_V1.csv', './MIT-BIH/114_MLII.csv', './MIT-BIH/114_V5.csv', './MIT-BIH/115_MLII.csv', './MIT-BIH/115_V1.csv', './MIT-BIH/116_MLII.csv', './MIT-BIH/116_V1.csv', './MIT-BIH/117_MLII.csv', './MIT-BIH/117_V2.csv', './MIT-BIH/118_MLII.csv', './MIT-BIH/118_V1.csv', './MIT-BIH/119_MLII.csv', './MIT-BIH/119_V1.csv', './MIT-BIH/121_MLII.csv', './MIT-BIH/121_V1.csv', './

In [5]:
train_folders = records[0: 76]
test_folders = records[:95]

In [6]:

rd = pd.read_csv(train_folders[1])

print(rd)

      0.752733  0.730747  0.159375  0.319714  0.266970  0.307124  0.270779  \
0     0.628595  0.571475 -0.009939  0.076968  0.122889  0.170717  0.142956   
1     0.735441  0.738430  0.084738  0.206169  0.121107  0.179561  0.122728   
2     0.738411  0.699388  0.051565  0.203068  0.125145  0.175656  0.134468   
3     0.680462  0.680163 -0.019243  0.161988  0.107638  0.159469  0.108530   
4     0.582148  0.524486 -0.040619  0.117758  0.091763  0.138946  0.111530   
...        ...       ...       ...       ...       ...       ...       ...   
2260  0.655886  0.462990 -0.027770  0.197068  0.131335  0.206258  0.154236   
2261  0.738342  0.627394 -0.061221  0.132494  0.047617  0.102561  0.056248   
2262  0.740587  0.666731  0.015605  0.212601  0.147614  0.204291  0.157438   
2263  0.677623  0.747006 -0.000209  0.125799  0.126878  0.156325  0.127174   
2264  0.766317  0.636956  0.035152  0.263598  0.155145  0.242336  0.164824   

      0.285674  0.281439  0.283470  ...  0.000000.62  0.000000.

In [7]:
# Đối với tập train
# Tạo dữ liệu train từ cửa sổ trượt
X_train, y_train, X_test, y_test = [], [], [], []

for i in train_folders:
    data = pd.read_csv(i, usecols=lambda column: column != 187) #record[0]

    #Chia khung
    window_input = 44
    window_out = 44

    # Số lượng lặp qua dữ liệu
    num_samples_train = len(data) - window_input - window_out + 1

    for i in range(num_samples_train):
        X_window = data[i:i+window_input]
        y_window = data[i+window_input:i+window_input+window_out]

        X_train.append(X_window)
        y_train.append(y_window)


In [9]:
for i in test_folders:
    data = pd.read_csv(i, usecols=lambda column: column != 187) #record[0]

    #Chia khung
    window_input = 44
    window_out = 44

    # Số lượng lặp qua dữ liệu
    num_samples_test = len(data) - window_input - window_out + 1

    # Đối với tập test
    for i in range(num_samples_test):
        X_window = data[i:i+window_input]
        y_window = data[i+window_input:i+window_input+window_out]

        X_test.append(X_window)
        y_test.append(y_window)

In [10]:
X_train= np.array(X_train)
y_train= np.array(y_train)
X_test= np.array(X_test)
y_test= np.array(y_test)

MemoryError: Unable to allocate 8.57 GiB for an array with shape (139037, 44, 188) and data type float64

In [ ]:
# Tạo một mô hình LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(10, 187)))  # Đặt input_shape phù hợp với kích thước của mảng X_train
model.add(Dense(5 * 187))  # Đặt lớp Dense phù hợp với kích thước của mảng y_train

# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')